In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
import scipy as sp
from sklearn import metrics
from bayesian_player_ratings import *

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

Load Match Summary Data

In [2]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary['Season'] = match_summary['Match_ID'].apply(lambda x: int(x[:4]))

player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats = create_additional_player_stats_variables(player_stats)
player_stats['exp_vaep_value'] = player_stats['exp_vaep_value'].fillna(0)
player_stats['exp_offensive_value'] = player_stats['exp_offensive_value'].fillna(0)
player_stats['exp_defensive_value'] = player_stats['exp_defensive_value'].fillna(0)

match_stats = aggregate_player_to_match_stats(player_stats)

match_summary_stats = match_summary.merge(match_stats, how = "left", on = "Match_ID")

Home Advantage

In [ ]:
home_average_value = match_summary_stats[match_summary_stats['Season']==2021]['Home_exp_vaep_value'].mean()
away_average_value = match_summary_stats[match_summary_stats['Season']==2021]['Away_exp_vaep_value'].mean()
home_advantage = home_average_value - away_average_value
home_advantage

Player Values

- Initialise every value at 0

In [ ]:
starting_player_values = player_stats[player_stats['Season'] == 2021].groupby(['Player', 'Team']).mean()[['exp_vaep_value']].reset_index()
starting_player_values['exp_vaep_value'] = 0

In [ ]:
starting_player_stds = player_stats[player_stats['Season'] == 2021].groupby('Player').std()[['exp_vaep_value']].reset_index()
starting_player_stds.columns = ['Player', 'exp_vaep_value_std']
starting_player_stds['exp_vaep_value_std'] = 10

Match Level Player Ratings & Team Ratings & Updating

1. Team Offensive Values are the summation of the player values 
2. Team Defensive Values are the average exp_vaep_value allowed against the team
3. Team Ratings are the Team Offensive Value - Team Defensive Value (Net Rating per game)
4. Match probabilities and predictions rely on the difference in ratings: RatingA - RatingB
5. Actual match takes place and players all accumulate their exp_vaep_value
6. Actual Offensive Values are the summation of individual players' values
7. Actual Rating for each team is the Net difference in values between each Team's Offensive Value
8. Players' values are updated (New Team Offensive Value calculated)
9. Teams' Ratings are updated (New Team Rating calculated)
10. Team's New Defensive Value is calculated as New Team Offensive Value - New Team Rating

In [ ]:
match_id = "202101_BrisbaneLions_Sydney"
match_player_stats = player_stats[player_stats['Match_ID'] == match_id]

home_players = list(match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']]['Player'].unique())
away_players = list(match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']]['Player'].unique())

home_player_values = starting_player_values[starting_player_values['Player'].isin(home_players)]
away_player_values = starting_player_values[starting_player_values['Player'].isin(away_players)]

In [ ]:
# Initialise all values at 0
home_offensive_value = 0
away_offensive_value = 0

home_defensive_value = 0
away_defensive_value = 0

In [ ]:
# Calculate team ratings as net Off - Def
home_rating = home_offensive_value - home_defensive_value
home_rating_ha = home_rating + home_advantage
away_rating = away_offensive_value - away_defensive_value

In [ ]:
# Rating Difference for calculating expected probabilities (from Home POV)
home_prior_rating_diff = home_rating_ha - away_rating
away_prior_rating_diff = away_rating - home_rating_ha
home_prior_rating_diff, away_prior_rating_diff

Turn Team Ratings into Match Predictions

In [ ]:
def calculate_probabilities(rating_diff, std):
    
    diff = stats.norm(loc = rating_diff, scale = std)
    
    away_prob = diff.cdf(0)
    draw_prob = diff.pdf(0)
    home_prob = 1 - away_prob - draw_prob
    
    return home_prob, draw_prob, away_prob

In [ ]:
home_prob, draw_prob, away_prob = calculate_probabilities(home_prior_rating_diff, 35)
home_prob, draw_prob, away_prob

Get Actual Match Result

In [ ]:
home_actual_player_values = match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']][['Player', 'exp_vaep_value']]
away_actual_player_values = match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']][['Player', 'exp_vaep_value']]

In [ ]:
home_actual_offensive_rating_ha = home_actual_player_values['exp_vaep_value'].sum()
home_actual_offensive_rating = home_actual_offensive_rating_ha - home_advantage
away_actual_offensive_rating = away_actual_player_values['exp_vaep_value'].sum()
home_actual_rating_diff = home_actual_offensive_rating_ha - away_actual_offensive_rating
away_actual_rating_diff = away_actual_offensive_rating - home_actual_offensive_rating_ha

home_actual_offensive_rating_ha, away_actual_offensive_rating, home_actual_rating_diff, away_actual_rating_diff

In [ ]:
home_rating_diff_error = home_actual_rating_diff - home_prior_rating_diff
away_rating_diff_error = away_actual_rating_diff - away_prior_rating_diff
home_rating_diff_error, away_rating_diff_error

Bayesian Value Updating

In [ ]:
def calculate_posterior(prior_mean, actual, prior_std=10, prior_weight = 1):
        
    return (prior_weight*(prior_std**2)*prior_mean + (2-prior_weight)*(prior_std**2)*actual) / (prior_std**2 + prior_std**2)

Calculate New Home Rating (Difference between Offensive Value and Defensive Value)

In [ ]:
home_posterior_rating_diff_ha = calculate_posterior(home_prior_rating_diff, home_actual_rating_diff, prior_std=10, prior_weight = 1.5)
new_home_rating = home_posterior_rating_diff_ha - home_advantage
home_prior_rating_diff, home_actual_rating_diff, home_posterior_rating_diff_ha, new_home_rating

Calculate New Offensive Value

In [ ]:
home_team = home_player_values.merge(home_actual_player_values, how = 'left', on = 'Player')
home_team.columns = ['Player', 'Team', 'prior', 'actual']
home_team['posterior'] = calculate_posterior(prior_mean=home_team['prior'], actual=home_team['actual'], prior_std=10, prior_weight = 1.5)
new_home_offensive_value = home_team['posterior'].sum()
new_home_offensive_value

Calculate New Defensive Value

In [ ]:
new_home_defensive_value = new_home_offensive_value - new_home_rating
new_home_defensive_value

Home Updated Ratings / Values

In [ ]:
new_home_offensive_value, new_home_defensive_value, new_home_rating

Away Team

In [ ]:
new_away_rating = calculate_posterior(away_prior_rating_diff, away_actual_rating_diff, prior_std=10, prior_weight = 1.5)
away_prior_rating_diff, away_actual_rating_diff, new_away_rating

In [ ]:
away_team = away_player_values.merge(away_actual_player_values, how = 'left', on = 'Player')
away_team.columns = ['Player', 'Team', 'prior', 'actual']
away_team['posterior'] = calculate_posterior(prior_mean=away_team['prior'], actual=away_team['actual'], prior_std=10, prior_weight = 1.5)
new_away_offensive_value = away_team['posterior'].sum()
new_away_offensive_value

In [ ]:
new_away_defensive_value = new_away_offensive_value - new_away_rating
new_away_defensive_value

In [ ]:
new_away_offensive_value, new_away_defensive_value, new_away_rating

Re-Factor Code

In [3]:
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
match_summary_stats_2021 = match_summary_stats[match_summary_stats['Season'] == 2021]

In [4]:
ratings = TeamRatings()
ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.initialise_player_values(player_stats_2021)
ratings.initialise_team_values(player_stats_2021)
# ratings.calculate_home_advantage(match_summary_stats_2021)
ratings.latest_round

'Start'

Round 202101

In [5]:
round_id = '202101'
# match_id = '202101_Collingwood_WesternBulldogs'

In [6]:
ratings.update_round_values(player_stats, round_id)

202101_BrisbaneLions_Sydney
202101_Collingwood_WesternBulldogs
202101_Essendon_Hawthorn
202101_GreaterWesternSydney_StKilda
202101_Melbourne_Fremantle
202101_NorthMelbourne_PortAdelaide
202101_Richmond_Carlton
202101_WestCoast_GoldCoast


In [ ]:
ratings.player_values['202101'].sort_values('Value', ascending = False).head(10)

In [7]:
ratings.team_values['202101']

,Team,Rating,Offensive_Value,Defensive_Value
0,Adelaide,0.0000,0.0000,0.0000
1,Brisbane Lions,-5.1272,0.0000,5.1272
2,Carlton,-9.4501,0.0000,9.4501
3,Collingwood,-6.7112,0.0000,6.7112
4,Essendon,0.1454,0.0000,-0.1454
5,Fremantle,-7.4888,3.2287,10.7176
6,Geelong,0.0000,0.0000,0.0000
7,Gold Coast,-6.7339,0.0000,6.7339
8,Greater Western Sydney,3.9049,0.0000,-3.9049
9,Hawthorn,-0.1454,2.0315,2.1769
